## Input sequence: "Dream big and work for it"

In [1]:
import torch

inputs = torch.tensor(
    [[0.72, 0.45, 0.31], # Dream    (x^1)
     [0.75, 0.20, 0.55], # big      (x^2)
     [0.30, 0.80, 0.40], # and      (x^3)
     [0.85, 0.35, 0.60], # work     (x^4)
     [0.55, 0.15, 0.75], # for      (x^5)
     [0.25, 0.20, 0.85]] # it       (x^6)
)

# Corresponding words
words = ['Dream', 'big', 'and', 'work', 'for', 'it']

## Class for implementing causal attention

In [2]:
from torch import nn

class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ values.transpose(1, 2)
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5, dim=-1
        )

        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

In [3]:
d_in = inputs.shape[-1]
d_out = 2
print(d_in, d_out)

3 2


In [5]:
batch = torch.stack((inputs,), dim=0)
print(batch.shape)

torch.Size([1, 6, 3])


## Class for implementing multi-head attention

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList([
            CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)
        ])

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [7]:
torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = 3, 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

In [8]:
context_vecs = mha(batch)
print(context_vecs)
print('context_vecs.shape:', context_vecs.shape)

tensor([[[-0.5762, -0.1627,  0.5569,  0.3635],
         [-0.5650, -0.0622,  0.5600,  0.2991],
         [-0.5474, -0.1223,  0.5293,  0.3414],
         [-0.5786, -0.0938,  0.5637,  0.3375],
         [-0.5586, -0.0418,  0.5521,  0.3015],
         [-0.5271, -0.0006,  0.5282,  0.2698]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([1, 6, 4])
